In [1]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData
from pyspark.sql.functions import col,explode

stockdata = GetStockData() 
spark = DevSparkSession().spark


ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 22:01:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
json_data  = stockdata.get_historical_stock_data()


data saved for symbol QQQ and date range 2021-01-01 to 2021-12-31.
data saved for symbol VOO and date range 2021-01-01 to 2021-12-31.


In [4]:
print(json_data[0].get("meta", {}).get("symbol"))

QQQ


In [56]:

df = spark.createDataFrame(json_data) 

df = df.withColumn("symbol", col("meta").symbol).filter( col("status") == "ok" ).drop(df.status)

df.show()

+--------------------+--------------------+------+
|                meta|              values|symbol|
+--------------------+--------------------+------+
|{exchange_timezon...|[{volume -> 23850...|   QQQ|
|{exchange_timezon...|[{volume -> 37518...|   VOO|
+--------------------+--------------------+------+



In [58]:

df.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy( "symbol" ) \
    .save("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze")


25/04/01 00:33:37 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1061216 ms exceeds timeout 120000 ms
25/04/01 00:33:37 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/01 00:42:09 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [ ]:
df = spark.read.load("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"); 
 

# profile data
df.limit(1).withColumn("values", col("values")[0]).show(truncate = False)



# Dimentional modelling 

Dim Date

Dim 
exchange_timezone
symbol
mic_code
NASDAQ 
currency
ETF 
__BusinessKeyHash ( track scd, used to identify if its same instance)
__Hash1Type ( based on bkhash - columns to overwrite if they changes)
__Hash2Type (based on bkhash - if hash doest match insert new row ) 
__CurrentFlag 
__DeletedFlag (0 - dont do for now)
__EffectiveStartDateTime
__EffectiveEndDateTime
__CreatedBatchLogID
__UpdatedBatchlogID
__CreateDateTime
__UpdateDateTime



Dim Stage
exchange_timezone
symbol
mic_code
NASDAQ 
currency
ETF 
__BusinessKeyHash ( track scd, used to identify if its same instance)
__Hash1Type ( based on bkhash - columns to overwrite if they changes) 
__Hash2Type (based on bkhash - if hash doest match insert new row ) 
__DeletedFlag (0 - dont do for now)
__CreatedDateTime ( current_timestamp())




Fact
volume
high
datetime
low
close
open 
__FactKeyHash (dedublication) (definse set of row they uniquily indetify row)
__DeletedFlag
__EffectiveStartDateTime
__EffectiveEndDateTime
__CreatedBatchLogID
__UpdatedBatchlogID
__CreateDateTime
__UpdateDateTime 


Fact Stage
volume
high
datetime
low
close
open 
__FactKeyHash (dedublication) (definse set of row they uniquily indetify row)
__DeletedFlag
__CreateDateTime current_timestamp())





 

+--------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+------+
|meta                                                                                                                                        |values                                                                                                               |symbol|
+--------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+------+
|{exchange_timezone -> America/New_York, symbol -> QQQ, mic_code -> XNMS, exchange -> NASDAQ, currency -> USD, interval -> 1day, type -> ETF}|{volume -> 23850100, high -> 403.57001, datetime -> 20

In [ ]:

df_flat_exploded = df_flat2.select(
                        col("symbol"),
                        explode( "values").alias("record"))
                        
                        # .withColumn( "volume" , col("record").getItem("volume") ) \
                        # .withColumn( "high" , col("record").getItem("high") ) \
                        # .withColumn( "low" , col("record").getItem("low") ) \
                        # .withColumn( "close" , col("record").getItem("close") ) \
                        # .withColumn( "open" , col("record").getItem("open") ) \
                        # .withColumn( "date" , col("record").getItem("datetime") ) \
                        #         .drop("record")


df_flat_exploded =   df_flat_exploded.select(
        "symbol",
        col("record").getItem("volume").alias("volume"),
        col("record").getItem("high").alias("high"),
        col("record").getItem("low").alias("low"),
        col("record").getItem("close").alias("close"),
        col("record").getItem("open").alias("open"),
        col("record").getItem("date").alias("date")
        
  )   
df_flat_exploded = df_flat_exploded.filter(  col("symbol")  == "VOO" ).distinct().count()     


print(df_flat_exploded)
# df_flat_exploded.show(truncate = False )

251
